In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# TODO: change to your driver path
driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
data_list = []

# TODO: change page #, fetch data from 2021 ~ 2023
# Option: save data in the mid of for-loop to avoid lossing data because of extremely long run time.
for page in range (1, 3):
    driver.get("https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={}".format(page))
    # Option: tune wait time
    wait = WebDriverWait(driver, 3)
    elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

    for element in elements:
        data_dict = {}
        try:
            notes = element.find_element_by_xpath('.//h6/span').text
            data_dict["Notes"] = notes

            university_program = element.find_element_by_xpath('.//h6').text
            data_dict["University & Program"] = university_program[:len(university_program) - len(notes)]

            date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
            data_dict["Date Added"] = date_added

            badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

            decision_and_date = badges[0].text
            data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]
            data_dict["Decision Date"] = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '')

            data_dict["Entrance Semester"] = badges[1].text

            data_dict["Degree"] = badges[-2].text

            keys = ["GPA", "GRE V", "GRE AW", "GRE"]
            for i in range(2, len(badges) - 2):
                for key in keys:
                    if key in badges[i].text:
                        data_dict[key] = badges[i].text.replace(key + ' ', '')
                        break

            data_list.append(data_dict)
        except Exception as e:
            continue

driver.quit()

# TODO: change data format of dates
# TODO: match col names from data to our dataset cols
# TODO: store data to .csv
for data in data_list:
    print(data)

{'Notes': '', 'University & Program': 'Forensic Psychology, John Jay College Of Criminal Justice', 'Date Added': 'May 10, 2023', 'Decision': 'Accepted', 'Decision Date': '10 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Masters', 'GPA': '3.50'}
{'Notes': 'Will not be accepting offer! Hope this helps others status be updated', 'University & Program': 'Community Psychology, DePaul University ', 'Date Added': 'May 08, 2023', 'Decision': 'Accepted', 'Decision Date': '8 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Masters', 'GPA': '3.80'}
{'Notes': '', 'University & Program': 'OISE - Counselling Psychology, University Of Toronto', 'Date Added': 'May 05, 2023', 'Decision': 'Rejected', 'Decision Date': '5 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Masters', 'GPA': '3.70'}
{'Notes': '(International student), graduated from one of highest rank uni in my country, already secured scholarship, and have several local publication', 'University & Program': 'Psychology, New York Uni

# Add multithread

In [1]:
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to fetch data from a specific page
def fetch_data(page):
    driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
    data_list = []

    driver.get(f"https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={page}")

    wait = WebDriverWait(driver, 2) #330
    elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

    for element in elements:
        data_dict = {}
        try:
            notes = element.find_element_by_xpath('.//h6/span').text
            data_dict["Notes"] = notes

            university_program = element.find_element_by_xpath('.//h6').text
            data_dict["University & Program"] = university_program[:len(university_program) - len(notes)]

            date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
            data_dict["Date Added"] = date_added

            badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

            decision_and_date = badges[0].text
            data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]
            data_dict["Decision Date"] = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '')

            data_dict["Entrance Semester"] = badges[1].text

            data_dict["Degree"] = badges[-2].text

            keys = ["GPA", "GRE V", "GRE AW", "GRE"]
            for i in range(2, len(badges) - 2):
                for key in keys:
                    if key in badges[i].text:
                        data_dict[key] = badges[i].text.replace(key + ' ', '')
                        break

            data_list.append(data_dict)
        except Exception as e:
            continue

    driver.quit()
    return data_list

# Define the range of pages to fetch
page_range = range(1, 3)  # Change this to include all 330 pages

# Fetch data from all pages in parallel with a maximum of 10 threads at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(fetch_data, page_range))

# Flatten the results
data_list = [item for sublist in results for item in sublist]

# TODO: change data format of dates
# TODO: match col names from data to our dataset cols
# TODO: store data to .csv
for data in data_list:
    print(data)


{'Notes': '', 'University & Program': 'MA Counselling Psychology, Simon Fraser University', 'Date Added': 'May 11, 2023', 'Decision': 'Accepted', 'Decision Date': '4 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Masters'}
{'Notes': '', 'University & Program': 'Forensic Psychology, John Jay College Of Criminal Justice', 'Date Added': 'May 10, 2023', 'Decision': 'Accepted', 'Decision Date': '10 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Masters', 'GPA': '3.50'}
{'Notes': 'Will not be accepting offer! Hope this helps others status be updated', 'University & Program': 'Community Psychology, DePaul University ', 'Date Added': 'May 08, 2023', 'Decision': 'Accepted', 'Decision Date': '8 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Masters', 'GPA': '3.80'}
{'Notes': '', 'University & Program': 'OISE - Counselling Psychology, University Of Toronto', 'Date Added': 'May 05, 2023', 'Decision': 'Rejected', 'Decision Date': '5 May', 'Entrance Semester': 'Fall 2023', 'Degree': 'Mas

In [1]:
import concurrent.futures
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to fetch data from a specific page
def fetch_data(page):
    driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
    data_list = []

    driver.get(f"https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={page}")

    wait = WebDriverWait(driver, 2) #330
    elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

    for element in elements:
        data_dict = {}
        try:
            notes = element.find_element_by_xpath('.//h6/span').text
            data_dict["Notes"] = notes

            university_program = element.find_element_by_xpath('.//h6').text
            data_dict["University & Program"] = university_program[:len(university_program) - len(notes)]

            date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
            data_dict["Date Added"] = date_added

            badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

            decision_and_date = badges[0].text
            data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]
            data_dict["Decision Date"] = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '')

            data_dict["Entrance Semester"] = badges[1].text

            data_dict["Degree"] = badges[-2].text

            keys = ["GPA", "GRE V", "GRE AW", "GRE"]
            for i in range(2, len(badges) - 2):
                for key in keys:
                    if key in badges[i].text:
                        data_dict[key] = badges[i].text.replace(key + ' ', '')
                        break

            data_list.append(data_dict)
        except Exception as e:
            continue

    driver.quit()
    return data_list

# Define the range of pages to fetch
page_range = range(1, 3)

# Fetch data from all pages in parallel with a maximum of 10 threads at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(fetch_data, page_range))

# Flatten the results
data_list = [item for sublist in results for item in sublist]

# Save data to a CSV file
with open('gradcafe_data.csv', mode='w', newline='') as csvfile:
    fieldnames = ['University & Program', 'Notes', 'Date Added', 'Decision', 'Decision Date', 'Entrance Semester', 'Degree', 'GPA', 'GRE V', 'GRE AW', 'GRE']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

# Match Data column

In [12]:
import concurrent.futures
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to fetch data from a specific page
def fetch_data(page):
    driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
    data_list = []

    driver.get(f"https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={page}")

    wait = WebDriverWait(driver, 2) #330
    elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

    for element in elements:
        data_dict = {}
        try:
            notes = element.find_element_by_xpath('.//h6/span').text
            data_dict["Notes"] = notes

            university_program = element.find_element_by_xpath('.//h6').text.split(', ')
            data_dict["University"] = university_program[0]
            data_dict["Program"] = university_program[1] if len(university_program) > 1 else ''

            date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
            data_dict["Date Added"] = date_added

            badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

            decision_and_date = badges[0].text
            data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]
            data_dict["Decision Date"] = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '')

            data_dict["Entrance Semester"] = badges[1].text

            data_dict["Degree"] = badges[-2].text

            keys = ["GPA", "GRE V", "GRE AW", "GRE"]
            for i in range(2, len(badges) - 2):
                for key in keys:
                    if key in badges[i].text:
                        new_key = {"GPA": "GPA", "GRE V": "GRE Verbal", "GRE AW": "GRE Writing", "GRE": "GRE Subject"}[key]
                        data_dict[new_key] = badges[i].text.replace(key + ' ', '')
                        break

            data_list.append(data_dict)
        except Exception as e:
            continue

    driver.quit()
    return data_list

# Define the range of pages to fetch
page_range = range(1, 3)

# Fetch data from all pages in parallel with a maximum of 10 threads at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(fetch_data, page_range))

# Flatten the results
data_list = [item for sublist in results for item in sublist]

# Save data to a CSV file
with open('gradcafe_data.csv', mode='w', newline='') as csvfile:
    fieldnames = ['University', 'Program', 'Degree', 'Entrance Semester', 'Decision', 'Decision Date', 'GPA', 'GRE Verbal', 'GRE Quantitive', 'GRE Writing', 'GRE Subject', 'Status', 'Date Added', 'Notes']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)


# match col names from data to our dataset cols and match date


In [1]:
import concurrent.futures
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to fetch data from a specific page
def fetch_data(page):
    driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
    data_list = []

    driver.get(f"https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={page}")

    wait = WebDriverWait(driver, 10) #330
    elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

    for element in elements:
        data_dict = {}
        try:
            notes = element.find_element_by_xpath('.//h6/span').text
            data_dict["Notes"] = notes

            h6_text = element.find_element_by_xpath('.//h6').text
            university_program = h6_text.replace(notes, '').strip().split(', ')
            data_dict["University"] = university_program[1] if len(university_program) > 1 else ''
            data_dict["Program"] = university_program[0]

            date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
            date_added_parts = date_added.split()
            data_dict["Date Added"] = f'{date_added_parts[1].replace(",", "")} {date_added_parts[0]} {date_added_parts[2]}'


            badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

            decision_and_date = badges[0].text
            data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]

            data_dict["Entrance Semester"] = badges[1].text
            decision_date_parts = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '').split()
            decision_year = data_dict["Entrance Semester"].split()[-1]
            data_dict["Decision Date"] = f'{decision_date_parts[0]} {decision_date_parts[1]} {decision_year}'

            data_dict["Degree"] = badges[-2].text

            keys = ["GPA", "GRE V", "GRE AW", "GRE"]
            for i in range(2, len(badges) - 2):
                for key in keys:
                    if key in badges[i].text:
                        new_key = {"GPA": "GPA", "GRE V": "GRE Verbal", "GRE AW": "GRE Writing", "GRE": "GRE Subject"}[key]
                        data_dict[new_key] = badges[i].text.replace(key + ' ', '')
                        break

            data_list.append(data_dict)
        except Exception as e:
            continue

    driver.quit()
    return data_list

# Define the range of pages to fetch 1 - 25
page_range = range(1, 331)

# Fetch data from all pages in parallel with a maximum of 10 threads at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(fetch_data, page_range))

# Flatten the results
data_list = [item for sublist in results for item in sublist]

# Save data to a CSV file
with open('gradcafe_data1.csv', mode='w', newline='') as csvfile:
    fieldnames = ['University', 'Program', 'Degree', 'Entrance Semester', 'Decision', 'Decision Date', 'GPA', 'GRE Verbal', 'GRE Quantitive', 'GRE Writing', 'GRE Subject', 'Status', 'Date Added', 'Notes']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

TimeoutException: Message: timeout: Timed out receiving message from renderer: 217.834
  (Session info: chrome=113.0.5672.92)


In [3]:
import concurrent.futures
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def save_to_csv(data_list, filename):
    with open(filename, mode='w', newline='') as csvfile:
        fieldnames = ['University', 'Program', 'Degree', 'Entrance Semester', 'Decision', 'Decision Date', 'GPA', 'GRE Verbal', 'GRE Quantitive', 'GRE Writing', 'GRE Subject', 'Status', 'Date Added', 'Notes']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for data in data_list:
            writer.writerow(data)

def fetch_data(page):
    driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
    data_list = []

    try:
        driver.get(f"https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={page}")

        wait = WebDriverWait(driver, 10)
        elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

        for element in elements:
            data_dict = {}
            try:
                notes = element.find_element_by_xpath('.//h6/span').text
                data_dict["Notes"] = notes

                h6_text = element.find_element_by_xpath('.//h6').text
                university_program = h6_text.replace(notes, '').strip().split(', ')
                data_dict["University"] = university_program[1] if len(university_program) > 1 else ''
                data_dict["Program"] = university_program[0]

                date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
                date_added_parts = date_added.split()
                data_dict["Date Added"] = f'{date_added_parts[1].replace(",", "")} {date_added_parts[0]} {date_added_parts[2]}'


                badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

                decision_and_date = badges[0].text
                data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]

                data_dict["Entrance Semester"] = badges[1].text
                decision_date_parts = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '').split()
                decision_year = data_dict["Entrance Semester"].split()[-1]
                data_dict["Decision Date"] = f'{decision_date_parts[0]} {decision_date_parts[1]} {decision_year}'

                data_dict["Degree"] = badges[-2].text

                keys = ["GPA", "GRE V", "GRE AW", "GRE"]
                for i in range(2, len(badges) - 2):
                    for key in keys:
                        if key in badges[i].text:
                            new_key = {"GPA": "GPA", "GRE V": "GRE Verbal", "GRE AW": "GRE Writing", "GRE": "GRE Subject"}[key]
                            data_dict[new_key] = badges[i].text.replace(key + ' ', '')
                            break

                data_list.append(data_dict)
            except Exception as e:
                continue

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        driver.quit()
        return data_list

# Define the range of pages to fetch 1 - 25
page_range = range(1, 50)

# Fetch data from all pages in parallel with a maximum of 10 threads at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(fetch_data, page_range))

# Flatten the results
data_list = [item for sublist in results for item in sublist]

# Save data to a CSV file
save_to_csv(data_list, 'gradcafe_data.csv')

Error occurred: Message: timeout: Timed out receiving message from renderer: 298.167
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 297.996
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 298.359
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 298.385
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 298.555
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 270.093
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 274.471
  (Session info: chrome=113.0.5672.92)

Error occurred: Message: timeout: Timed out receiving message from renderer: 287.295
  (Session info: chrome=113.0.5672.92)



KeyboardInterrupt: 

Correct Code

In [1]:

import concurrent.futures
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def save_to_csv(data_list, filename):
    with open(filename, mode='a', newline='') as csvfile:
        fieldnames = ['University', 'Program', 'Degree', 'Entrance Semester', 'Decision', 'Decision Date', 'GPA', 'GRE Verbal', 'GRE Quantitive', 'GRE Writing', 'GRE Subject', 'Status', 'Date Added', 'Notes']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        for data in data_list:
            writer.writerow(data)

def fetch_data(page):
    driver = webdriver.Chrome(executable_path='/Users/alanhsieh/Desktop/drivers/chromedriver')
    data_list = []

    try:
        driver.get(f"https://www.thegradcafe.com/survey/?per_page=40&q=psychology&institution=&program=&degree=&page={page}")

        wait = WebDriverWait(driver, 10)
        elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col')))

        for element in elements:
            data_dict = {}
            try:
                notes = element.find_element_by_xpath('.//h6/span').text
                data_dict["Notes"] = notes

                h6_text = element.find_element_by_xpath('.//h6').text
                university_program = h6_text.replace(notes, '').strip().split(', ')
                data_dict["University"] = university_program[1] if len(university_program) > 1 else ''
                data_dict["Program"] = university_program[0]

                date_added = element.find_element_by_xpath('.//p').text.replace('Added on ', '')
                date_added_parts = date_added.split()
                data_dict["Date Added"] = f'{date_added_parts[1].replace(",", "")} {date_added_parts[0]} {date_added_parts[2]}'


                badges = element.find_elements_by_xpath('.//div[@class="mt-3"]/span')

                decision_and_date = badges[0].text
                data_dict["Decision"] = decision_and_date[:decision_and_date.find(' ')]

                data_dict["Entrance Semester"] = badges[1].text
                decision_date_parts = decision_and_date[decision_and_date.find(' ') + 1:].replace('on ', '').split()
                decision_year = data_dict["Entrance Semester"].split()[-1]
                data_dict["Decision Date"] = f'{decision_date_parts[0]} {decision_date_parts[1]} {decision_year}'

                data_dict["Degree"] = badges[-2].text

                keys = ["GPA", "GRE V", "GRE AW", "GRE"]
                for i in range(2, len(badges) - 2):
                    for key in keys:
                        if key in badges[i].text:
                            new_key = {"GPA": "GPA", "GRE V": "GRE Verbal", "GRE AW": "GRE Writing", "GRE": "GRE Subject"}[key]
                            data_dict[new_key] = badges[i].text.replace(key + ' ', '')
                            break

                data_list.append(data_dict)
            except Exception as e:
                continue

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        driver.quit()
        return data_list


# Initialize an empty list for storing data from all pages
all_data = []

# Create a CSV file with headers
with open('gradcafe_data.csv', mode='w', newline='') as csvfile:
    fieldnames = ['University', 'Program', 'Degree', 'Entrance Semester', 'Decision', 'Decision Date', 'GPA', 'GRE Verbal', 'GRE Quantitive', 'GRE Writing', 'GRE Subject', 'Status', 'Date Added', 'Notes']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

# Define the range of pages to fetch 1 - 50
page_range = range(1, 3)

# Fetch data from all pages in parallel with a maximum of 10 threads at a time
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    for i in range(0, len(page_range), 10):
        results = list(executor.map(fetch_data, page_range[i:i+10]))

        # Flatten the results
        data_list = [item for sublist in results for item in sublist]

        # Save data to a CSV file
        save_to_csv(data_list, 'gradcafe_data.csv')

        # Append the data_list to the all_data list
        all_data.extend(data_list)
